https://datahack.analyticsvidhya.com/contest/janatahack-independence-day-2020-ml-hackathon/#ProblemStatement

-----------------------------------------------------------------------------------------------------
https://github.com/hiteshsharmanew/ML-hackathons-/blob/master/NLP_final_script.py

-----------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import tensorflow as tf 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
ss=pd.read_csv('sample_submission_UVKGLZE.csv')

In [3]:
train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [4]:
test.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [5]:
ss.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,0,0,0
1,20974,0,0,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,0,0,0,0,0,0


In [6]:
print(train.shape)
print(test.shape)

(20972, 9)
(8989, 3)


In [7]:
train_target=train.iloc[:,3:]

In [8]:
train_target.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0


In [9]:
train_text=train['TITLE']+' '+train['ABSTRACT']
test_text=test['TITLE']+' '+test['ABSTRACT']

In [10]:
train_text.head()

0    Reconstructing Subject-Specific Effect Maps   ...
1    Rotation Invariance Neural Network   Rotation ...
2    Spherical polyharmonics and Poisson kernels fo...
3    A finite element approximation for the stochas...
4    Comparative study of Discrete Wavelet Transfor...
dtype: object

In [11]:
test_text.head()

0    Closed-form Marginal Likelihood in Gamma-Poiss...
1    Laboratory mid-IR spectra of equilibrated and ...
2    Case For Static AMSDU Aggregation in WLANs   F...
3    The $Gaia$-ESO Survey: the inner disk intermed...
4    Witness-Functions versus Interpretation-Functi...
dtype: object

In [12]:
all_text=pd.concat([train_text,test_text])

# Preprocessing the Text

In [13]:
def preprocess(text):
    processed_text =text.str.replace(r'\d+(\.\d+)?', 'numbr')
    # Remove punctuation
    processed_text = processed_text.str.replace(r'[^\w\d\s]', ' ')
    
    # Replace whitespace between terms with a single space
    processed_text = processed_text.str.replace(r'\s+', ' ')
    
    # Remove leading and trailing whitespace
    processed_text = processed_text.str.replace(r'^\s+|\s+?$', '')
    
    # change words to lower case - Hello, HELLO, hello are all the same word
    processed_text = processed_text.str.lower()
    
    return processed_text


In [14]:
processed_text=preprocess(all_text)

C:\Users\Admin\AppData\Local\Temp/ipykernel_7868/1320096268.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_text =text.str.replace(r'\d+(\.\d+)?', 'numbr')
C:\Users\Admin\AppData\Local\Temp/ipykernel_7868/1320096268.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_text = processed_text.str.replace(r'[^\w\d\s]', ' ')
C:\Users\Admin\AppData\Local\Temp/ipykernel_7868/1320096268.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_text = processed_text.str.replace(r'\s+', ' ')
C:\Users\Admin\AppData\Local\Temp/ipykernel_7868/1320096268.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_text = processed_text.str.replace(r'^\s+|\s+?$', '')


In [15]:
processed_text.head()

0    reconstructing subject specific effect maps pr...
1    rotation invariance neural network rotation in...
2    spherical polyharmonics and poisson kernels fo...
3    a finite element approximation for the stochas...
4    comparative study of discrete wavelet transfor...
dtype: object

# Model Prepration


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

word_vectorizer=TfidfVectorizer(min_df=1,smooth_idf=True,norm='l2',tokenizer=lambda x:x.split(),sublinear_tf=True,
                               ngram_range=(1,3))

word_vectorizer.fit(processed_text)

C:\Users\Admin\anaconda3\envs\with tensorflow\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True,
                tokenizer=<function <lambda> at 0x0000023BA7477790>)

In [19]:
train_word_features=word_vectorizer.transform(train_text)

test_word_features = word_vectorizer.transform(test_text)

In [20]:
print("After vectorising this is the shape of train",train_word_features.shape)
print("After vectorising this is the shape of test",test_word_features.shape)

After vectorising this is the shape of train (20972, 4444910)
After vectorising this is the shape of test (8989, 4444910)


In [21]:
train_features=train_word_features
test_features=test_word_features

In [24]:
#Model preprations 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

classifier=OneVsRestClassifier(LinearSVC(penalty="l2",loss='hinge',class_weight = "balanced"), n_jobs=-1)

classifier.fit(train_features, train_target)
predictions=classifier.predict(test_features)

In [25]:
predictions=classifier.predict(test_features)

In [26]:
## submission conversion        
submission=pd.DataFrame(predictions, columns=['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance'])
submission=pd.concat([test['ID'],submission],axis=1)
submission.to_csv("ovr_full_balanced_script_final.csv", index=False)
print(submission.shape)
submission.head()

(8989, 7)


,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [28]:
## Checking for f1 score 

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(train_features,train_target,test_size=0.1)

print(c.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(18874, 4444910)
(2098, 4444910)
(18874, 6)
(2098, 6)


In [30]:
classifier.fit(x_train, y_train)
predictions=classifier.predict(x_test)

In [32]:
from sklearn.metrics import classification_report

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       858
           1       0.93      0.85      0.89       600
           2       0.83      0.80      0.81       577
           3       0.80      0.77      0.78       517
           4       1.00      0.02      0.04        45
           5       1.00      0.04      0.07        27

   micro avg       0.84      0.81      0.82      2624
   macro avg       0.90      0.56      0.57      2624
weighted avg       0.85      0.81      0.81      2624
 samples avg       0.83      0.84      0.81      2624



C:\Users\Admin\anaconda3\envs\with tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
